In [1]:
%load_ext autoreload
%autoreload 2

# Model Analyzer



In [ ]:
import os
import sys

import pandas as pd

os.chdir(os.path.expanduser("~/Desktop/causeDMS"))
sys.path.append(os.path.expanduser("~/Desktop/causeDMS/src/"))

from cosmos import DMSData, PriorFactory, ModelBuilder, ModelAnalyzer

In [3]:
# Load the data

pheno1, pheno2 = "abundance", "surface2"
no_s_hat = False
s_hat_flag = "no_s_hat" if no_s_hat else "s_hat"

df = pd.read_csv(f"test/data/kir21_effect_{pheno1}_{pheno2}.csv")

data_obj = DMSData(
    df,
    [pheno1, pheno2],
    include_type=["missense"],
    exclude_type=[
        "synonymous",
        "insertion1",
        "insertion2",
        "insertion3",
        "deletion1",
        "deletion2",
        "deletion3",
    ],
    min_num_variants_per_group=10,
)
prior = PriorFactory(data_obj)
prior.gen_prior(
    x_name="beta_hat_1",
    y_name="beta_hat_2",
    x_se_name="se_hat_1",
    x_gmm_n_components=2,
)
model = ModelBuilder(prior, "test/data/results/")

Un-comment the following cell if you have not run it already in the last notebook.

In [4]:
# for i in range(1, 10):  # Limit the number of groups for testing
#     print(f"Running position {i}...", end="\r")
#     model.run_cosmos(
#         i,
#         no_s_hat=no_s_hat,
#         suppress_pareto_warning=True,
#     )

The analyzer will automatically pick up the results from the `model`'s `path_directory` (in this case, `"test/data/results/"`).

In [5]:
analyzer = ModelAnalyzer(model, "test/data/results/analysis/", False)

For each position, the summary table generates:
- `group`: The group that the position belongs to. Positions with insufficient number of variants will be merged into the same group. Refer to `DMSData` construction in `00_dms_data.ipynb` for details.
- `model_rank{i}`: The `i`-th best model. (By default `i=1`)
- `{parameter}_mean`: The mean estimate of the parameter. If `NaN`, the parameter is not in the model.
- `{parameter}_std`: The error estimate of the parameter. If `NaN`, the parameter is not in the model.

In [6]:
df_sum = analyzer.summary(rank=1, save=True)  # The summary table will be saved to the data_path
df_sum

,position,group,model_rank1,tau_mean,tau_std,gamma_mean,gamma_std
0,2,1,model_5,NaN,NaN,0.629284,0.275893
1,3,2,model_5,NaN,NaN,0.717805,0.254125
2,4,3,model_6,0.474483,0.278360,0.680578,0.448206
3,5,4,model_6,0.475656,0.284296,1.253374,0.405198
4,6,5,model_6,0.413907,0.274230,0.736984,0.391255
5,7,6,model_6,0.464315,0.278957,0.734544,0.315520
6,8,7,model_6,0.301579,0.265880,0.924676,0.414763
7,9,8,model_6,1.227650,0.404221,1.112426,0.414872
8,10,9,model_5,NaN,NaN,0.486677,0.305256
